In [1]:
import numpy as np
import pandas as pd
import utils
from nn import NeuralNetwork
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
def evaluate_classification_metrics(y_true, y_pred, multiclass=False):

    average_type = 'macro' if multiclass else 'binary'

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average=average_type, zero_division=0)
    rec = recall_score(y_true, y_pred, average=average_type, zero_division=0)
    f1 = f1_score(y_true, y_pred, average=average_type, zero_division=0)

    print(f"✅ Accuracy : {acc:.4f}")
    print(f"✅ Precision: {prec:.4f}")
    print(f"✅ Recall   : {rec:.4f}")
    print(f"✅ F1 Score : {f1:.4f}")

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1
    }


In [3]:
# Load the dat
train_data = pd.read_csv(
    'data/loan-10k.lrn.csv'
)

train_data.head()


,ID,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,...,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year,grade
0,24341,12500.0,12500.0,12500.0,36 months,7.21,387.17,< 1 year,MORTGAGE,81000.0,...,N,6,2018,6,2000,2,2019,2,2019,A
1,67534,33850.0,33850.0,33775.0,60 months,20.99,915.57,1 year,MORTGAGE,80000.0,...,N,10,2015,9,1984,2,2019,2,2019,E
2,35080,10000.0,10000.0,10000.0,60 months,20.00,264.94,< 1 year,RENT,36580.0,...,N,9,2017,10,2006,1,2018,11,2018,D
3,4828,20250.0,20250.0,20250.0,36 months,14.31,695.15,9 years,RENT,48700.0,...,N,0,2015,6,1996,6,2016,9,2017,C
4,59259,25000.0,25000.0,25000.0,36 months,14.99,866.52,1 year,MORTGAGE,85000.0,...,N,11,2016,0,2002,2,2019,2,2019,C


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# 1. Separate features and target
X = train_data.drop(columns=["ID", "grade"])
y = train_data["grade"]

# 2. Label encode target (A, B, C → 0, 1, 2)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 3. One-hot encode target for softmax
ohe_y = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
y_onehot = ohe_y.fit_transform(y_encoded.reshape(-1, 1))

# 4. Identify feature types
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = [col for col in X.columns if col not in cat_cols]

# 5. Preprocess features: One-hot categorical + scale numeric
preprocessor = ColumnTransformer(transformers=[
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", StandardScaler(), num_cols)
])

X_processed = preprocessor.fit_transform(X)

# 6. Train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y_onehot, test_size=0.2, random_state=42
)


In [5]:
configs = [
    # Simple
    {"layers": [X_train.shape[1],y_train.shape[1]], "act": [utils.softmax], "lr": 0.01},
    {"layers": [X_train.shape[1], y_train.shape[1]], "act": [utils.softmax], "lr": 0.1},

    # 1 hidden layer
    {"layers": [X_train.shape[1], 16, y_train.shape[1]], "act": [utils.relu, utils.softmax], "lr": 0.01},
    {"layers": [X_train.shape[1], 16, y_train.shape[1]], "act": [utils.tanh, utils.softmax], "lr": 0.01},
    {"layers": [X_train.shape[1], 16, y_train.shape[1]], "act": [utils.relu, utils.softmax], "lr": 0.1},
    {"layers": [X_train.shape[1], 16, y_train.shape[1]], "act": [utils.tanh, utils.softmax], "lr": 0.1},

    # 2 hidden layers
    {"layers": [X_train.shape[1], 16, 8, y_train.shape[1]], "act": [utils.relu, utils.relu, utils.softmax], "lr": 0.01},
    {"layers": [X_train.shape[1], 16, 8, y_train.shape[1]], "act": [utils.tanh, utils.relu, utils.softmax], "lr": 0.01},
    {"layers": [X_train.shape[1], 16, 8, y_train.shape[1]], "act": [utils.tanh, utils.tanh, utils.softmax], "lr": 0.01},
    {"layers": [X_train.shape[1], 16, 8, y_train.shape[1]], "act": [utils.relu, utils.relu, utils.softmax], "lr": 0.1},
    {"layers": [X_train.shape[1], 16, 8, y_train.shape[1]], "act": [utils.tanh, utils.relu, utils.softmax], "lr": 0.1},
    {"layers": [X_train.shape[1], 16, 8, y_train.shape[1]], "act": [utils.tanh, utils.tanh, utils.softmax], "lr":0.1}]


In [6]:
results = []

for i, cfg in enumerate(configs):
    print(f"\n🔁 Running config {i+1}/{len(configs)}")
    print(f"Layers: {cfg['layers']}, Activations: {[fn.__name__ for fn in cfg['act']]}, LR: {cfg['lr']}")

    nn = NeuralNetwork(cfg["layers"], cfg["act"], learning_rate=cfg["lr"],multiclass=True)
    
    # Train including validation set
    nn.train(X_train, y_train, epochs=100, X_val=X_val, y_val=y_val)
    
    # Predict final output
    preds = nn.predict(X_val)  # output: class labels
    y_val_labels = np.argmax(y_val, axis=1)  # one-hot → class index

    metrics = evaluate_classification_metrics(y_val_labels, preds, multiclass=True)

    
    # Get metrics stored during training from the model
    train_loss = nn.history["train_loss"][-1]
    val_loss = nn.history["val_loss"][-1] if nn.history["val_loss"] else None
    train_acc = nn.history["train_acc"][-1]
    val_acc = nn.history["val_acc"][-1] if nn.history["val_acc"] else None


    results.append({
        "config_id": i + 1,
        "layers": cfg["layers"],
        "activations": [fn.__name__ for fn in cfg["act"]],
        "learning_rate": cfg["lr"],

        # Add train history
        "train_loss": float(train_loss),
        "val_loss": float(val_loss) if val_loss is not None else None,
        "train_acc": float(train_acc),
        "val_acc": float(val_acc) if val_acc is not None else None,

        # Add evaluation metrics from evaluate_classification_metrics()
        **metrics})




🔁 Running config 1/12
Layers: [179, 7], Activations: ['softmax'], LR: 0.01
Epoch 1/100, Loss: 2.7666, Acc: 0.4308, Val Loss: 1.4462, Val Acc: 0.5410
Epoch 2/100, Loss: 1.0855, Acc: 0.6081, Val Loss: 1.0561, Val Acc: 0.6335
Epoch 3/100, Loss: 0.8257, Acc: 0.6870, Val Loss: 0.8887, Val Acc: 0.6785
Epoch 4/100, Loss: 0.6934, Acc: 0.7298, Val Loss: 0.8009, Val Acc: 0.7080
Epoch 5/100, Loss: 0.6202, Acc: 0.7575, Val Loss: 0.7354, Val Acc: 0.7325
Epoch 6/100, Loss: 0.5695, Acc: 0.7796, Val Loss: 0.6975, Val Acc: 0.7445
Epoch 7/100, Loss: 0.5309, Acc: 0.7930, Val Loss: 0.6588, Val Acc: 0.7510
Epoch 8/100, Loss: 0.5068, Acc: 0.8063, Val Loss: 0.6386, Val Acc: 0.7590
Epoch 9/100, Loss: 0.4840, Acc: 0.8145, Val Loss: 0.6153, Val Acc: 0.7705
Epoch 10/100, Loss: 0.4602, Acc: 0.8210, Val Loss: 0.5974, Val Acc: 0.7795
Epoch 11/100, Loss: 0.4516, Acc: 0.8275, Val Loss: 0.5822, Val Acc: 0.7810
Epoch 12/100, Loss: 0.4328, Acc: 0.8327, Val Loss: 0.5714, Val Acc: 0.7885
Epoch 13/100, Loss: 0.4280, Acc: 

In [8]:
df_results = pd.DataFrame(results)
df_results

,config_id,layers,activations,learning_rate,train_loss,val_loss,train_acc,val_acc,accuracy,precision,recall,f1_score
0,1,"[179, 7]",[softmax],0.01,0.270317,0.427863,0.892375,0.8405,0.8405,0.714092,0.706593,0.707552
1,2,"[179, 7]",[softmax],0.10,1.057552,1.660113,0.852125,0.8250,0.8250,0.700871,0.710071,0.700474
2,3,"[179, 16, 7]","[relu, softmax]",0.01,0.269263,0.601914,0.903375,0.8520,0.8520,0.716525,0.746271,0.701078
3,4,"[179, 16, 7]","[tanh, softmax]",0.01,0.227352,0.700322,0.916625,0.7710,0.7710,0.563622,0.572372,0.567573
4,5,"[179, 16, 7]","[relu, softmax]",0.10,1.600814,1.657420,0.292250,0.2930,0.2930,0.041920,0.142614,0.064794
5,6,"[179, 16, 7]","[tanh, softmax]",0.10,0.401251,0.729021,0.852625,0.7900,0.7900,0.623656,0.646522,0.622865
6,7,"[179, 16, 8, 7]","[relu, relu, softmax]",0.01,0.346954,0.683483,0.878625,0.7810,0.7810,0.604245,0.537451,0.555477
7,8,"[179, 16, 8, 7]","[tanh, relu, softmax]",0.01,0.251210,0.744486,0.902875,0.7820,0.7820,0.574360,0.607085,0.577096
8,9,"[179, 16, 8, 7]","[tanh, tanh, softmax]",0.01,0.384089,0.845680,0.849125,0.6920,0.6920,0.463394,0.490017,0.474423
9,10,"[179, 16, 8, 7]","[relu, relu, softmax]",0.10,1.602039,1.608216,0.292000,0.2935,0.2935,0.041929,0.142857,0.064830
